# Data Preprocessing

author: Melih Kacaman
date: 05-10-2022

# Constants

In [1]:
import warnings
warnings.filterwarnings("ignore")

DATASET1_ODBC = 'PROTO.Melih.Dataset1_SimilarProducts'
DATASET1_TITLE = 'dataset1'

## Connect to the MSSQL Server

In [2]:
from common.sql_helper import custom_query
import pandas as pd

query = f'select * from {DATASET1_ODBC}'
result = custom_query(query=query)

result.head()

,WebSecenek,MarkaKodu,CinsiyetKodu,UrunGrubuKodu,Renk,cUrl,UrlNo
0,50053296-850,08,2,016,850,https://tedarik.aydinli.com.tr/SupplierPortal/...,2
1,50053303-999,08,2,016,999,https://tedarik.aydinli.com.tr/SupplierPortal/...,1
2,50053303-999,08,2,016,999,https://tedarik.aydinli.com.tr/SupplierPortal/...,2
3,50053321-001,08,2,016,001,https://tedarik.aydinli.com.tr/SupplierPortal/...,1
4,50053321-001,08,2,016,001,https://tedarik.aydinli.com.tr/SupplierPortal/...,2


# Download Images

In [3]:
class_names = result[['CinsiyetKodu', 'UrunGrubuKodu', 'Renk']].drop_duplicates()
class_names.head()

,CinsiyetKodu,UrunGrubuKodu,Renk
0,2,016,850
1,2,016,999
3,2,016,001
5,2,016,430
9,2,016,750


In [4]:
import requests

def save_image(image_url, path):
    img_data = requests.get(image_url).content
    with open(path, 'wb') as handler:
        handler.write(img_data)

In [5]:
import os

def make_dataset(bran_id, dataset_name):
    try:
        base = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '.'))
        path = os.path.join('datasets', dataset_name)
        path = os.path.join(base, path)
        if not os.path.isdir(path):
            os.mkdir(path)
        path = os.path.join(path, f'brand_{bran_id}')
        if not os.path.isdir(path):
            os.mkdir(path)
        # datasets/dataset/brand_id

        for index, row in class_names.iterrows():
            _class = row['CinsiyetKodu'] + '-'+ row['UrunGrubuKodu'] + '-' + row['Renk']
            folder = os.path.join(path, _class)
            res = result.query(f'MarkaKodu == "{bran_id}" and CinsiyetKodu == "{row.CinsiyetKodu}" and UrunGrubuKodu == "{row.UrunGrubuKodu}" and Renk == "{row.Renk}"')

            # if exists any data
            if res.shape[0] > 0:
                # check the folder exist
                if not os.path.isdir(folder):
                    os.mkdir(folder)

                # add the imgs to the folder
                for img_index, img_row in res.iterrows():
                    save_image(img_row['cUrl'], os.path.join(folder, str(img_index) + '.jpg'))
    except Exception as e:
        print('An exception occurred.', e)


In [6]:
%%time
res = make_dataset('08', dataset_name=DATASET1_TITLE)

KeyboardInterrupt: 

## For USPA

## For PC

## For Cacharel